In [7]:
import gym
import numpy as np

env = gym.make("MountainCar-v0")
state = env.reset()
DISCRETE_OS_SIZE = [20, 20]
discrete_os_win_size = (env.observation_space.high - env.observation_space.low)/DISCRETE_OS_SIZE
q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))

print(discrete_os_win_size)

done = False
while not done:
    action = 2
    new_state, reward, done, _ = env.step(action)
    print(reward, new_state)
    env.render()
env.close()

[0.09  0.007]
-1.0 [-4.29151361e-01  3.03268662e-04]
-1.0 [-0.42854701  0.00060435]
-1.0 [-0.42764592  0.00090109]
-1.0 [-0.42645458  0.00119134]
-1.0 [-0.42498156  0.00147303]
-1.0 [-0.42323742  0.00174414]
-1.0 [-0.42123467  0.00200275]
-1.0 [-0.41898764  0.00224703]
-1.0 [-0.41651237  0.00247526]
-1.0 [-0.41382652  0.00268586]
-1.0 [-0.41094915  0.00287737]
-1.0 [-0.40790066  0.00304849]
-1.0 [-0.4047026   0.00319807]
-1.0 [-0.40137746  0.00332514]
-1.0 [-0.39794858  0.00342888]
-1.0 [-0.39443992  0.00350866]
-1.0 [-0.3908759   0.00356403]
-1.0 [-0.3872812  0.0035947]
-1.0 [-0.38368062  0.00360058]
-1.0 [-0.38009888  0.00358174]
-1.0 [-0.37656046  0.00353843]
-1.0 [-0.3730894   0.00347106]
-1.0 [-0.3697092  0.0033802]
-1.0 [-0.3664426  0.0032666]
-1.0 [-0.36331149  0.00313111]
-1.0 [-0.36033674  0.00297475]
-1.0 [-0.35753809  0.00279866]
-1.0 [-0.354934    0.00260408]
-1.0 [-0.35254162  0.00239239]
-1.0 [-0.35037658  0.00216503]
-1.0 [-0.34845303  0.00192356]
-1.0 [-0.34678345  0.00

KeyboardInterrupt: 